<p style='direction:rtl; text-align: right'>ابتدا باید کتابخانه های زیر را وارد کنیم:
    <ul style='direction:rtl; text-align: right'>
        <li>numpy: برای کار با ماتریس ها</li>
        <li>pandas: برای کار با فایل</li>
        <li>string: برای کار با رشته</li>
        <li>nltk: برای کار با متن</li>
        <li>networkx: برای کار با گراف</li>
        <li>scipy.sparse: برای کار با ماتریس تنک</li>
    </ul>
</p>

In [1]:
import numpy as np
import pandas as pd
import string
import nltk
import networkx as nx
from scipy.sparse import find

<h1 style='direction:rtl; text-align: right'>
قسمت ۱: استفاده از Random Walk برای پیشنهاد کلمه
</h1>
<p style='direction:rtl; text-align: right'>در این پروژه میخواهیم یک الگوریتم پیشنهاد کلمه بر اساس گشت تصادفی طراحی کنیم. </p>

<h2 style='direction:rtl; text-align: right'>
قسمت ۱.۱: خواندن فایل imdb_labelled.txt
</h2>
<p style='direction:rtl; text-align: right'>از تابع read_csv کتابخانه <a href="https://www.geeksforgeeks.org/python-read-csv-using-pandas-read_csv/">pandas</a> استفاده کنید.</p>
<p style='direction:rtl; text-align: right'>حتما حتما از آرگومان های sep و names استفاده شود.</p>

In [2]:
sep = "\t"
names=['sentence', 'label']


df = pd.read_csv("imdb_labelled.txt", names=names, sep=sep)


df.head()

,sentence,label
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


<h2 style='direction:rtl; text-align: right'>
قسمت ۱.۲: استخراج ستون مربوط به جمله ها
</h2>

In [3]:
data = df['sentence'].values
print(data)

['A very, very, very slow-moving, aimless movie about a distressed, drifting young man.  '
 'Not sure who was more lost - the flat characters or the audience, nearly half of whom walked out.  '
 'Attempting artiness with black & white and clever camera angles, the movie disappointed - became even more ridiculous - as the acting was poor and the plot and lines almost non-existent.  '
 'Very little music or anything to speak of.  '
 'The best scene in the movie was when Gerardo is trying to find a song that keeps running through his head.  '
 "The rest of the movie lacks art, charm, meaning... If it's about emptiness, it works I guess because it's empty.  "
 'Wasted two hours.  '
 'Saw the movie today and thought it was a good effort, good messages for kids.  '
 'A bit predictable.  '
 'Loved the casting of Jimmy Buffet as the science teacher.  '
 'And those baby owls were adorable.  '
 "The movie showed a lot of Florida at it's best, made it look very appealing.  "
 'The Songs Were The 

<h2 style='direction:rtl; text-align: right'>
قسمت ۱.۳: پیش پردازش داده های متنی
</h2>
<p style='direction:rtl; text-align: right'>برای پیش پردازش داده های متنی باید روند زیر را طی کرد:
    <ul style='direction:rtl; text-align: right'>
        <li>نوشتن کلمات با حروف کوچک با استفاده از تابع lower زیرا در غیر اینصورت متن ما حساس به بزرگی و کوچکی حروف می شود.</li>
        <li>حذف space ها اضافه با استفاده از تابع strip</li>
        <li>حذف تمام علامت های نگارشی با استفاده از تابع translate زیرا هیچ ارزشی ندارند.</li>
        <li>جدا کردن کلمات موجود در جملات با استفاده از تابع word_tokenize تا بتوانیم با کلمات کار کنیم.</li>  
    </ul>
</p>

In [4]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\esmaeilzadeh_PC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
new_data = []
for i in data:
    temp = i.strip().lower()
    new_data.append(nltk.word_tokenize(temp.translate(str.maketrans('', '', string.punctuation))))
print(new_data)

[['a', 'very', 'very', 'very', 'slowmoving', 'aimless', 'movie', 'about', 'a', 'distressed', 'drifting', 'young', 'man'], ['not', 'sure', 'who', 'was', 'more', 'lost', 'the', 'flat', 'characters', 'or', 'the', 'audience', 'nearly', 'half', 'of', 'whom', 'walked', 'out'], ['attempting', 'artiness', 'with', 'black', 'white', 'and', 'clever', 'camera', 'angles', 'the', 'movie', 'disappointed', 'became', 'even', 'more', 'ridiculous', 'as', 'the', 'acting', 'was', 'poor', 'and', 'the', 'plot', 'and', 'lines', 'almost', 'nonexistent'], ['very', 'little', 'music', 'or', 'anything', 'to', 'speak', 'of'], ['the', 'best', 'scene', 'in', 'the', 'movie', 'was', 'when', 'gerardo', 'is', 'trying', 'to', 'find', 'a', 'song', 'that', 'keeps', 'running', 'through', 'his', 'head'], ['the', 'rest', 'of', 'the', 'movie', 'lacks', 'art', 'charm', 'meaning', 'if', 'its', 'about', 'emptiness', 'it', 'works', 'i', 'guess', 'because', 'its', 'empty'], ['wasted', 'two', 'hours'], ['saw', 'the', 'movie', 'today'

<h2 style='direction:rtl; text-align: right'>
قسمت ۱.۴: ساخت راس های گراف جهت دار با استفاده از کلمات استخراج شده
</h2>
<p style='direction:rtl; text-align: right'>برای این کار ابتدا باید یک گراف جهت دار با استفاده از <a href="https://networkx.org/documentation/stable/tutorial.html">networkx</a> بسازیم.</p>
<p style='direction:rtl; text-align: right'>سپس با استفاده از تابع add_nodes_from یا add_node نود های گراف را اضافه می کنیم.</p>
<p style='direction:rtl; text-align: right'>از کلمات موجود در new_data استفاده کنید</p>
<p style='direction:rtl; text-align: right'>توجه شود نود های تکراری یکی در نظر گرفته می شوند در نتیجه نگران تکراری بودن نود ها نباشید.</p>

In [6]:
G = nx.DiGraph()
for i in new_data:
    G.add_nodes_from(i)
print('Number of nodes=',len(G.nodes))

Number of nodes= 3173


<h2 style='direction:rtl; text-align: right'>
قسمت ۱.۵: اضافه کردن یال های گراف بر اساس ترتیب قرار گیری کلمات
</h2>
<p style='direction:rtl; text-align: right'>برای این کار باید یکی یکی جملات را پیمایش کنیم و بین هر دو کلمه کنار هم یک یال وزن دار اضافه کنیم. به طور مثال جمله:</p>
<p style='direction:rtl; text-align: right'>i am a student</p>
<p style='direction:rtl; text-align: right'>به ترتیب یک یال از i به am و یک یال از am به a و یک یال از a به student اضافه میکنیم.</p>
<p style='direction:rtl; text-align: right'>اگر دوباره یکی از کلمات بعد از دیگری امد وزن آن یال را اضافه می کنیم.</p>


In [7]:
for i in range(len(new_data)):
    for j in range(len(new_data[i])-1):
        try:
            G[new_data[i][j]][new_data[i][j+1]]["weight"] += 1
        except:
            G.add_edge(new_data[i][j], new_data[i][j+1],weight=1)

<h2 style='direction:rtl; text-align: right'>
قسمت ۱.۵: ساخت ماتریس مجاورت از روی گراف
</h2>
<p style='direction:rtl; text-align: right'>با استفاده از تابع adjacency_matrix موجود در  <a href="https://networkx.org/documentation/stable/tutorial.html">networkx</a> ماتریس مجاورت گراف را بسازید. دقت کنید که خروجی یک ماتریس تنک است. یعنی فقط درایه های ناصفر ذخیره شده اند. </p>



In [8]:
A=nx.adjacency_matrix(G)
print(A)

  (0, 1)	14
  (0, 4)	20
  (0, 6)	1
  (0, 9)	1
  (0, 22)	2
  (0, 31)	2
  (0, 34)	1
  (0, 42)	1
  (0, 47)	4
  (0, 60)	2
  (0, 87)	13
  (0, 92)	3
  (0, 106)	7
  (0, 109)	1
  (0, 110)	1
  (0, 114)	1
  (0, 120)	1
  (0, 131)	2
  (0, 138)	3
  (0, 142)	1
  (0, 145)	1
  (0, 147)	11
  (0, 165)	1
  (0, 169)	3
  (0, 170)	1
  :	:
  (3146, 3147)	1
  (3147, 262)	1
  (3148, 3149)	1
  (3149, 326)	1
  (3150, 3151)	1
  (3151, 12)	1
  (3152, 3153)	1
  (3153, 90)	1
  (3154, 29)	1
  (3155, 3156)	1
  (3156, 410)	1
  (3158, 16)	1
  (3159, 3160)	1
  (3160, 32)	1
  (3161, 333)	1
  (3163, 3164)	1
  (3164, 1978)	1
  (3165, 571)	1
  (3166, 125)	1
  (3166, 271)	1
  (3168, 3166)	1
  (3169, 871)	1
  (3170, 54)	1
  (3171, 536)	1
  (3172, 581)	1


<h2 style='direction:rtl; text-align: right'>
قسمت ۱.۶: ایجاد یک mapping بین راسهای های گراف و اندیس موجود در ماتریس مجاورت
</h2>
<p style='direction:rtl; text-align: right'>از انجایی که اعضای ماتریس اسپارس همگی عدد هستند و نود های گراف ما همگی کلمات هستند باید این mapping انجام شود.</p>




In [9]:
g_nodes = list(G.nodes)
# print(g_nodes)
print(g_nodes[3]) # print 4th vertex

aimless


<h2 style='direction:rtl; text-align: right'>
قسمت ۱.۷:  تعریف تابع randomwalk 
</h2>
<p style='direction:rtl; text-align: right'>ورودی های این تابع:
    <ul style='direction:rtl; text-align: right'>
        <li>A: ماتریس مجاورت</li>
        <li>g_nodes: لیست رئوس</li>
        <li>k: تعداد تکرار</li>
        <li>initial: شماره راس شروع گشت</li>
    </ul>
</p>
<p style='direction:rtl; text-align: right'>خروجی این تابع: یک آرایه از k راس تصادفی مستقل از خروجی های راس شروع. دقت کنید که خروجی یک گشت به طول k نیست بلکه k راس تصادفی مجاور با راس شروع است.</p>
<p style='direction:rtl; text-align: right'>احتمال انتخاب یک مجاور با وزن یال مربوطه متناسب است.</p>

<p style='direction:rtl; text-align: right'>
راهنمایی: برای انتخاب تصادفی راس مجاور از دستور np.random.choice استفاده کنید. </p>

<p style='direction:rtl; text-align: right'>
راهنمایی: برای ساختن لیست مجاورهای یک راس از تابع find از بسته scipy.sparse بر روی ماتریس مجاورت استفاده کنید. </p>



In [172]:
def randwalk(A,g_nodes,k,initial):# Adjacency matrix, nodes list, no. of iterations, initial vertex
#     loc=initial
...
X=randwalk(A,g_nodes,3,1)
print('initial vertex=',g_nodes[1])
print('random neighbors=')
print(X)

initial vertex= very
random neighbors=
['limited', 'energetic', 'smart']


In [ ]:
def randwalk(A,g_nodes,k,initial):# Adjacency matrix, nodes list, no. of iterations, initial vertex
   # loc=initial
    for i in range(0,k):
        B=find(A([initial,:]))
        gfg=np.random.choise(B)
    return g_nodes(gfg)    
X=randwalk(A,g_nodes,3,1)
print('initial vertex=',g_nodes[1])
print('random neighbors=')
print(X)

In [17]:
initial=1
_,B,_=find(A[initial,:])
B

array([   1,    2,   36,   47,   87,  111,  116,  118,  138,  169,  222,
        238,  269,  305,  443,  505,  515,  529,  581,  590,  602,  650,
        685, 1061, 1140, 1248, 1284, 1311, 1312, 1324, 1332, 1404, 1508,
       1528, 1774, 1946, 2114, 2167, 2175, 2266, 2350, 2352, 2585])

<h2 style='direction:rtl; text-align: right'>
قسمت ۱.۸: پیشنهاد کلمه بعدی براساس رئوس پیمایش شده در گشت 
</h2>

<p style='direction:rtl; text-align: right'>
در این قسمت از کاربر یک کلمه را به عنوان کلمه شروع می گیریم (با دستور input). سپس براساس گشت تصادفی سه کلمه را به عنوان کلمه بعدی پیشنهاد می دهیم. کاربر می تواند یکی از این کلمه ها را انتخاب کند یا خودش یک کلمه جدید وارد کند یا پایان را بزند. سپس جمله نوشته می شود و این روند تا وقتی که پایان انتخاب شود ادامه می یابد.
</p>
<p style='direction:rtl; text-align: right'>توجه کنید که کلمات رئوس گراف با حروف کوچک هستند بنابراین باید با دستور lower ورودی را به حروف کوچک تبدیل کنید. همچنین برای پیدا کردن شماره راس با استفاده از کلمه مربوط به آن از دستور index استفاده کنید. مثلا g_nodes.index('see') شماره راس مربوط به کلمه see را برمیگرداند.</p>




In [196]:
k=3


Enter initial word:I
   1 despised
   2 enjoy
   3 hate
   4 None
   0 Finish
Choose a number:2
' I enjoy ...'
   1 this
   2 it
   3 visual
   4 None
   0 Finish
Choose a number:1
' I enjoy this ...'
   1 dribble
   2 subpar
   3 film
   4 None
   0 Finish
Choose a number:3
' I enjoy this film ...'
   1 just
   2 and
   3 it
   4 None
   0 Finish
Choose a number:4
Enter next word:very
' I enjoy this film very ...'
   1 restrained
   2 strong
   3 disappointed
   4 None
   0 Finish
Choose a number:4
Enter next word:much
' I enjoy this film very much ...'
   1 sucked
   2 of
   3 from
   4 None
   0 Finish
Choose a number:0
I enjoy this film very much
